In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import optuna
import joblib
from collections import namedtuple

In [2]:
Data = namedtuple('Data', ["type","name", "path", "config"])

In [3]:
DATA_PATH = Path("data")
OUTPUT_PATH = Path("output")
STUDY_PATH = Path("study")

In [4]:
POL_CLS_UUID = "66dc899c4c4f4e4f822d02bdddb18f80"

In [5]:
experiments = [
 'classification|default-of-credit-card-clients',
 'classification|heloc',
 'classification|eye_movements',
 'classification|Higgs',
 'classification|pol',
 'classification|albert',
 'classification|road-safety',
 'classification|MiniBooNE',
 'classification|covertype',
 'classification|jannis',
 'classification|Bioresponse',
 'regression|cpu_act',
 'regression|Ailerons',
 'regression|yprop_4_1',
 'regression|superconduct',
 'regression|Allstate_Claims_Severity',
 'regression|topo_2_1',
 'regression|Mercedes_Benz_Greener_Manufacturing',
]

# experiments = [Experiment(*d.split("|")) for d in experiments]

In [6]:
datasets = []
for experiment in experiments:
    typ, name = experiment.split("|")
    folder = DATA_PATH/name
    config_files = list(folder.glob("*config*"))
    config = np.load(config_files[0], allow_pickle=True).item()
    config["n_iter"] = len(config_files)
    datasets.append(Data(typ, name, folder, config))

In [7]:
dataset_dict = {d.name: d for d in datasets}

In [8]:
dataset_names = list(dataset_dict.keys())
dataset_names

['default-of-credit-card-clients',
 'heloc',
 'eye_movements',
 'Higgs',
 'pol',
 'albert',
 'road-safety',
 'MiniBooNE',
 'covertype',
 'jannis',
 'Bioresponse',
 'cpu_act',
 'Ailerons',
 'yprop_4_1',
 'superconduct',
 'Allstate_Claims_Severity',
 'topo_2_1',
 'Mercedes_Benz_Greener_Manufacturing']

In [9]:
sel_columns = [
 'Unnamed: 0',
 'benchmark',
 'data__keyword',
 '_timestamp',
 'model_name',
 'model_type',
 'one_hot_encoder',

 'mean_time',
 'std_time',

 'mean_test_score',

 'max_test_score',
 'min_test_score',
 'std_test_score',

 'test_scores',
 'times',
 ]

In [10]:
tuning_results = pd.DataFrame(columns=sel_columns)

In [11]:
def get_study(dataset_name):
    if dataset_name == "pol":
        return pd.read_csv(OUTPUT_PATH/f"study_pol_{POL_CLS_UUID}.csv")
    folder = OUTPUT_PATH
    outputs = [f for f in folder.glob("study*") if dataset_name in f.name]
    if len(outputs)==1:
        return pd.read_csv(outputs[0])
    else: # For cncelled and unfinished trials
        folder  = STUDY_PATH
        study_path = [f for f in folder.glob(f"*{dataset_name}*.db") if "full" not in f.name][0]
        study_name = study_path.name.split(".")[0]
        study = optuna.load_study(study_name=study_name, storage=f"sqlite:///{study_path}")
        return study.trials_dataframe()
    
def format_study(study, dataset):
    type = dataset_dict[dataset.name].type
    categorical = dataset_dict[dataset.name].config["data__categorical"]
    if categorical==1.0:
        benchmark = "categorical"
    else:
        benchmark = "numerical"
    benchmark+=f"_{type}_medium"
    study['benchmark'] = benchmark
    study["data__keyword"] = dataset.name
    # try:
    study["duration"] = pd.to_timedelta(study["duration"], errors="coerce")
    study["mean_time"] = study["duration"].dt.total_seconds()/dataset_dict[dataset.name].config["n_iter"]
    # except AttributeError as e:
    #     for i, row in study.iterrows():
    #         try:
    #             study.loc[i, "duration"].dt.total_seconds() 
    #         except AttributeError as e:
    #             print(row["duration"])
    #             raise e
    #     raise e
    rename_dict = {
        "datetime_start": "_timestamp",
        "value": "mean_test_score",
    }
    study.rename(columns=rename_dict, inplace=True)
    study["model_name"] = "GANDALF"
    study["model_type"] = "PyTorchTabular"
    param_cols = [c for c in study.columns if c.startswith("params_")]
    study['params']=study[param_cols].apply(dict, axis=1)
    study = study.loc[study.state=="COMPLETE"]
    intersection_cols = list(set(study.columns).intersection(set(sel_columns)))
    study = study.loc[:, intersection_cols+["params"]]
    return study
            

In [12]:
#Add row for each experiment
for dataset in datasets:
    print(dataset.name)
    study = get_study(dataset.name)
    study = format_study(study, dataset)
    tuning_results = pd.concat([tuning_results,study], ignore_index=True)

default-of-credit-card-clients
heloc
eye_movements
Higgs
pol
albert
road-safety
MiniBooNE
covertype
jannis
Bioresponse
cpu_act
Ailerons
yprop_4_1
superconduct
Allstate_Claims_Severity
topo_2_1
Mercedes_Benz_Greener_Manufacturing


In [13]:
tuning_results.head()

,Unnamed: 0,benchmark,data__keyword,_timestamp,model_name,model_type,one_hot_encoder,mean_time,std_time,mean_test_score,max_test_score,min_test_score,std_test_score,test_scores,times,params
0,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:41:46.516483,GANDALF,PyTorchTabular,NaN,14.715429,NaN,0.503480,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.4753571532049581, 'p..."
1,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:42:30.691539,GANDALF,PyTorchTabular,NaN,50.556374,NaN,0.682418,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.15212112147976886, '..."
2,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:45:02.387713,GANDALF,PyTorchTabular,NaN,30.633439,NaN,0.721277,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.08526206184364576, '..."
3,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:46:34.307562,GANDALF,PyTorchTabular,NaN,95.580769,NaN,0.662469,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.15585553804470548, '..."
4,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:51:21.070666,GANDALF,PyTorchTabular,NaN,45.122091,NaN,0.704700,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.13567451588694796, '..."


In [14]:
tuning_results["_timestamp"] = pd.to_datetime(tuning_results["_timestamp"])

In [15]:
tuning_results.to_parquet("data/tuning_results.parquet")

## Results Compilation

In [30]:
experiment_results = pd.read_csv(OUTPUT_PATH/"tuned_params_runs.csv")

In [31]:
experiment_results['model_name'] = "GANDALF"


In [32]:
def get_benchmark(x):
    type = dataset_dict[x].type
    categorical = dataset_dict[x].config["data__categorical"]
    if categorical==1.0:
        benchmark = "categorical"
    else:
        benchmark = "numerical"
    benchmark+=f"_{type}_medium"
    return benchmark

experiment_results['benchmark']=experiment_results['dataset'].apply(get_benchmark)

In [33]:
experiment_results.columns

Index(['Name', 'State', 'Notes', 'User', 'Tags', 'Created', 'Runtime', 'Sweep',
       'batch_size', 'dataset', 'early_stopping_patience', 'max_epochs',
       'n_folds', 'optimizer', 'wandb', 'max_test_score', 'mean_test_score',
       'mean_time', 'min_test_score', 'std_test_score', 'std_time',
       'test_scores', 'times', 'model_name', 'benchmark'],
      dtype='object')

In [34]:
cols = [
    "benchmark",
    "dataset",
    "model_name",
    "mean_test_score",
    "max_test_score",
    "min_test_score",
    "std_test_score",
    "mean_time",
    "std_time",
    "test_scores",
    "times",
    "Name"
]

In [35]:
experiment_results = experiment_results.loc[~experiment_results.Name.str.contains("dropout"), cols]
experiment_results = experiment_results.loc[~experiment_results.Name.str.contains("feature"), cols]

In [36]:
experiment_results.drop(columns=["Name"], inplace=True)

In [37]:
benchmark_results = pd.read_parquet("data/benchmark_best_rows.parquet")

In [38]:
benchmark_results = benchmark_results.reset_index().rename(columns={"data__keyword": "dataset"})

In [39]:
experiment_results = pd.concat([experiment_results, benchmark_results.reset_index()])
experiment_results

,benchmark,dataset,model_name,mean_test_score,max_test_score,min_test_score,std_test_score,mean_time,std_time,test_scores,times,index
0,numerical_regression_medium,Ailerons,GANDALF,0.864613,0.866160,0.863550,0.001119,11.039673,1.790702,"[0.8641285300254822,0.8661600947380066,0.86355...","[13.085770606994627,11.308923244476318,8.72432...",NaN
1,categorical_classification_medium,albert,GANDALF,0.685300,NaN,NaN,NaN,26.371789,NaN,NaN,NaN,NaN
3,categorical_regression_medium,Allstate_Claims_Severity,GANDALF,0.591058,NaN,NaN,NaN,202.469212,NaN,NaN,NaN,NaN
4,numerical_classification_medium,Bioresponse,GANDALF,0.881648,0.909280,0.849771,0.020634,6.990565,1.542113,"[0.8722430467605591,0.8497711420059204,0.89762...","[5.676318407058716,4.947674989700317,8.5587186...",NaN
5,categorical_classification_medium,covertype,GANDALF,0.932200,NaN,NaN,NaN,213.263477,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
128,numerical_regression_medium,yprop_4_1,MLP,0.022334,0.028097,0.027160,0.000389,9.533991,1.351060,"[0.0271595548838377, 0.028096886351704597, 0.0...","[11.402910470962524, 8.255438089370728, 8.9436...",128.0
129,numerical_regression_medium,yprop_4_1,RandomForest,0.093881,0.027303,0.026174,0.000479,4.994533,1.042345,"[0.026174030891650664, 0.027302874929199757, 0...","[6.466503143310547, 4.327132701873779, 4.18996...",129.0
130,numerical_regression_medium,yprop_4_1,Resnet,0.042830,0.028033,0.026744,0.000526,45.061062,6.024756,"[0.02674437314271927, 0.028033148497343063, 0....","[47.17852449417114, 36.85503840446472, 51.1496...",130.0
131,numerical_regression_medium,yprop_4_1,SAINT,0.059454,0.027719,0.026659,0.000437,180.976444,25.281317,"[0.026659198554938267, 0.0277189787766971, 0.0...","[207.14980340003967, 146.79630780220032, 188.9...",131.0


In [40]:
experiment_results.to_parquet("data/experiment_results_best_rows.parquet")

In [41]:
test_scores = pd.pivot_table(experiment_results.reset_index(), index=["benchmark", "dataset"], columns="model_name", values="mean_test_score")
# format test_scores df so that highest scores is highlighted
test_scores.style.highlight_max(axis=1, color='lightgreen')

In [42]:
test_std = pd.pivot_table(experiment_results.reset_index(), index=["benchmark", "dataset"], columns="model_name", values="std_test_score")
# format test_scores df so that lowest scores in each row is highlighted with a colormap
test_std.style.background_gradient(cmap='RdYlGn_r', axis=1)

In [43]:
mean_times = pd.pivot_table(experiment_results.reset_index(), index=["benchmark", "dataset"], columns="model_name", values="mean_time")
# format mean_times df with a red amber green color scale - reversed
mean_times.style.background_gradient(cmap='RdYlGn_r', axis=1)